# Getting the latent space of pre-trained models and predict task


## Importing the functions & libraries

In [1]:
import os
ml_code_path='/home/leilapirhaji/mz_embed_engine/ml'
os.chdir(ml_code_path)

import os
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
import optuna
import importlib
import random

import json

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader


from pretrain.get_pretrain_encoder import get_pretrain_input_data
from models.models_VAE import VAE
from pretrain.train_pretrain_VAE import PretrainVAE, pretrain_vae, optimize_hyperparameters

%who



[neptune] [warning] NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
2024-09-17 19:56:20.817678: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-17 19:56:21.049135: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-17 19:56:21.117448: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-17 19:56:21.604256: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions

DataLoader	 LabelEncoder	 PretrainVAE	 TensorDataset	 VAE	 get_pretrain_input_data	 importlib	 json	 label_encoder	 
ml_code_path	 nn	 np	 optim	 optimize_hyperparameters	 optuna	 os	 pd	 pretrain_vae	 
random	 torch	 


## Importing the pretrain input data

In [2]:
input_data_location='/home/leilapirhaji/PROCESSED_DATA_2'

# getting the input data
(X_data_all, y_data_all, X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test)=get_pretrain_input_data(input_data_location)

X_data_all.shape, y_data_all.shape, X_data_train.shape, y_data_train.shape, X_data_val.shape, y_data_val.shape, X_data_test.shape, y_data_test.shape

/home/leilapirhaji/mz_embed_engine/ml/pretrain/get_pretrain_encoder.py:72: DtypeWarning: Columns (1,8,28,30,31,32,33,34,46,50,51,52,53,54,55,56,57,58) have mixed types. Specify dtype option on import or set low_memory=False.
  y_data_all = pd.read_csv(pretrain_y_all, index_col=0)


((20548, 2736),
 (20548, 58),
 (14391, 2736),
 (14391, 58),
 (3074, 2736),
 (3074, 58),
 (3083, 2736),
 (3083, 58))

In [52]:
def parse_range_or_single(values, is_int=False):
    """
    Custom argparse type to parse either a single value or a range from a list.
    If `is_int` is True, parses integers; otherwise, parses floats.
    
    Example inputs: 
        ["256"] -> Outputs: 256 (int)
        ["256", "512", "32"] -> Outputs: (256, 512, 32) (int)
    """
    # If it's a list but contains only one element, treat it as a single value
    if len(values) == 1:
        if is_int:
            return int(values[0])  # Single integer value
        else:
            return float(values[0])  # Single float value
    
    # Otherwise, treat it as a range
    if is_int:
        return tuple(map(int, values))  # Convert to tuple of integers
    else:
        return tuple(map(float, values))  # Convert to tuple of floats





def main():
    
    # Set up the argument parser
    parser = argparse.ArgumentParser(description='Run VAE pretraining with hyperparameter optimization.')

    # Add arguments for the input directory and save directory
    parser.add_argument('--input_data_location', type=str, required=True, 
                        help='Location of the input data directory')
    parser.add_argument('--pretrain_save_dir', type=str, required=True, 
                        help='Directory to save the pre-trained models')

    # Add arguments for the hyperparameters
    parser.add_argument('--latent_size', nargs='*', default=[256],
                        help='Latent size: Either a single value or "min max step" for range (integer)')
    parser.add_argument('--num_hidden_layers', nargs='*', default=[2],
                        help='Number of hidden layers (either fixed or a range, integer)')
    parser.add_argument('--dropout_rate', nargs='*', default=[0.1], 
                        help='Dropout rate: Either a single value or "min max step" for range (float)')
    parser.add_argument('--learning_rate', nargs='*', default=["1e-5", "1e-2"], 
                        help='Learning rate range: Either a single value or "min max" for range (float)')
    parser.add_argument('--weight_decay', nargs='*', default=["1e-6", "1e-4"],
                        help='Weight decay range: Either a single value or "min max" for range (float)')
    parser.add_argument('--batch_size', nargs='*', default=[64],
                        help='Batch size (either fixed or a range, integer)')
    parser.add_argument('--patience', nargs='*', default=[10, 30, 5], 
                        help='Patience: Either a single value or "min max step" for range (integer), 0 means no early stopping')
    parser.add_argument('--num_epochs', nargs='*', default=[100], 
                        help='Number of epochs (either fixed or a range, integer)')
    parser.add_argument('--trial_name', type=str, default='vae_hyperparam_optimization_test', 
                        help='The name of this trial for Optuna study')
    parser.add_argument('--n_trials', type=int, default=20, 
                        help='Number of trials for hyperparameter optimization')

    print('Arguments are parsed')

    # Parse the arguments
    args = parser.parse_args()

    combined_params = {
        'latent_size': parse_range_or_single(args.latent_size, is_int=True),
        'num_hidden_layers': parse_range_or_single(args.num_hidden_layers, is_int=True),
        'dropout_rate': parse_range_or_single(args.dropout_rate, is_int=False),
        'learning_rate': parse_range_or_single(args.learning_rate, is_int=False),
        'weight_decay': parse_range_or_single(args.weight_decay, is_int=False),
        'batch_size': parse_range_or_single(args.batch_size, is_int=True),
        'patience': parse_range_or_single(args.patience, is_int=True),
        'num_epochs': parse_range_or_single(args.num_epochs, is_int=True)
    }

In [59]:
min_val, max_val, step = combined_params['latent_size']
min_val, max_val, step

(256, 512, 32)

## Pre-traing the models

### Training for a fixed paramters

In [9]:
import importlib
import pretrain.train_pretrain_VAE
importlib.reload(pretrain.train_pretrain_VAE)
from pretrain.train_pretrain_VAE import PretrainVAE, pretrain_vae, optimize_hyperparameters


# Combined parameter ranges and default values (with flexibility to tune)
# Combined parameter ranges and default values (with flexibility to tune)
combined_params = {
    'latent_size': 460, #(256, 512, 32),            # Min, Max, Step size (tuned)
    'num_hidden_layers': 2, # (2, 4, 1),          # Min, Max, Step size (tuned)
    'dropout_rate': 0.4, #(0.1, 0.5, 0.05),        # Min, Max, Step size (tuned)
    'learning_rate': (1e-5, 1e-4),           # Min, Max (log scale, tuned)
    'weight_decay': 1e-6, # (1e-6, 1e-3),            # Min, Max (log scale, tuned)

    # Training parameters that could also be tuned
    'batch_size': 64, #(32, 128, 32),             # Min, Max, Step size (tuned)
    'patience': 0, #(3, 10, 1),                  # Min, Max, Step size (tuned)
    'num_epochs': 10 # (10, 50, 5)                # Min, Max, Step size (tuned)
}

# Run hyperparameter optimization with model saving
trial_name = "vae_hyperparam_optimization_test"
pretrain_save_dir='/home/leilapirhaji/pretrained_models'
os.makedirs(pretrain_save_dir, exist_ok=True)

best_trial, study = optimize_hyperparameters(X_data_train, X_data_val, combined_params, trial_name, pretrain_save_dir, n_trials=3)


print(f"Best hyperparameters: {best_trial.params}")


[I 2024-09-17 20:45:32,445] A new study created in memory with name: no-name-4ee3c4a6-ba6a-43b5-944c-d101575040c6
/home/leilapirhaji/mz_embed_engine/ml/pretrain/train_pretrain_VAE.py:215: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  # Categorical parameter (e.g., activation functions, optimizer)


Epoch 1/10, Loss: 139175.6226
Validation Loss: 115.4729
Epoch 2/10, Loss: 2554010044920.5679
Validation Loss: 115.9617
Epoch 3/10, Loss: 1351940710536.1370
Validation Loss: 117.1198
Epoch 4/10, Loss: 1628550693946.8540
Validation Loss: 122.0877
Epoch 5/10, Loss: nan
Validation Loss: nan
Epoch 6/10, Loss: nan
Validation Loss: nan
Epoch 7/10, Loss: nan
Validation Loss: nan
Epoch 8/10, Loss: nan
Validation Loss: nan
Epoch 9/10, Loss: nan
Validation Loss: nan
Epoch 10/10, Loss: nan
Validation Loss: nan


[W 2024-09-17 20:45:54,963] Trial 0 failed with parameters: {'learning_rate': 8.175450568622918e-05} because of the following error: The value nan is not acceptable.
[W 2024-09-17 20:45:54,964] Trial 0 failed with value nan.


Encoder saved to /home/leilapirhaji/pretrained_models/vae_hyperparam_optimization_test/trial_0/encoder_state.pt
Hyperparameters saved to /home/leilapirhaji/pretrained_models/vae_hyperparam_optimization_test/trial_0/model_hyperparameters.json
Epoch 1/10, Loss: 115.8339
Validation Loss: 115.6603
Epoch 2/10, Loss: 115.9456
Validation Loss: 115.4980
Epoch 3/10, Loss: 117.0096
Validation Loss: 115.5116
Epoch 4/10, Loss: 121.4398
Validation Loss: 115.5286
Epoch 5/10, Loss: 530.2932
Validation Loss: 115.5630
Epoch 6/10, Loss: 16798847.7076
Validation Loss: 115.6314
Epoch 7/10, Loss: 163715.3808
Validation Loss: 115.6485
Epoch 8/10, Loss: 1262657.5379
Validation Loss: 115.7182
Epoch 9/10, Loss: 15166.6439
Validation Loss: 115.7463


[I 2024-09-17 20:46:17,703] Trial 1 finished with value: 115.77964611442722 and parameters: {'learning_rate': 3.29841897915774e-05}. Best is trial 1 with value: 115.77964611442722.


Epoch 10/10, Loss: 1103577.0938
Validation Loss: 115.7796
Encoder saved to /home/leilapirhaji/pretrained_models/vae_hyperparam_optimization_test/trial_1/encoder_state.pt
Hyperparameters saved to /home/leilapirhaji/pretrained_models/vae_hyperparam_optimization_test/trial_1/model_hyperparameters.json
Epoch 1/10, Loss: 117.0970
Validation Loss: 115.7322
Epoch 2/10, Loss: 115.8777
Validation Loss: 115.7385
Epoch 3/10, Loss: 115.9921
Validation Loss: 115.7424
Epoch 4/10, Loss: 116.0271
Validation Loss: 115.7421
Epoch 5/10, Loss: 115.8897
Validation Loss: 115.7337
Epoch 6/10, Loss: 115.8617
Validation Loss: 115.6954
Epoch 7/10, Loss: 115.7973
Validation Loss: 115.5866
Epoch 8/10, Loss: 115.7065
Validation Loss: 115.5406
Epoch 9/10, Loss: 115.7196
Validation Loss: 115.5387


[I 2024-09-17 20:46:40,161] Trial 2 finished with value: 115.53838208256936 and parameters: {'learning_rate': 1.6771169678691354e-05}. Best is trial 2 with value: 115.53838208256936.


Epoch 10/10, Loss: 115.6667
Validation Loss: 115.5384
Encoder saved to /home/leilapirhaji/pretrained_models/vae_hyperparam_optimization_test/trial_2/encoder_state.pt
Hyperparameters saved to /home/leilapirhaji/pretrained_models/vae_hyperparam_optimization_test/trial_2/model_hyperparameters.json
Best trial: FrozenTrial(number=2, state=TrialState.COMPLETE, values=[115.53838208256936], datetime_start=datetime.datetime(2024, 9, 17, 20, 46, 17, 704091), datetime_complete=datetime.datetime(2024, 9, 17, 20, 46, 40, 161711), params={'learning_rate': 1.6771169678691354e-05}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.0001, log=True, low=1e-05, step=None)}, trial_id=2, value=None)
Best hyperparameters: {'learning_rate': 1.6771169678691354e-05}
Best hyperparameters: {'learning_rate': 1.6771169678691354e-05}


In [11]:
study.best_trial.value, study.best_trial.params

(115.53838208256936, {'learning_rate': 1.6771169678691354e-05})

In [12]:
try:
    import optuna.visualization as vis
    vis.plot_optimization_history(study).show()
    vis.plot_param_importances(study).show()
except ImportError:
    print("Optuna visualization is not available. Install optuna[visualization] for plotting.")


### Optuna hyper-par optimization

In [46]:
import importlib
import pretrain.train_pretrain_VAE
importlib.reload(pretrain.train_pretrain_VAE)
from pretrain.train_pretrain_VAE import PretrainVAE, pretrain_vae, optimize_hyperparameters


# Combined parameter ranges and default values (with flexibility to tune)
# Combined parameter ranges and default values (with flexibility to tune)
combined_params = {
    'latent_size': (256, 512, 32),            # Min, Max, Step size (tuned)
    'num_hidden_layers': 2, # (2, 4, 1),          # Min, Max, Step size (tuned)
    'dropout_rate': (0.1, 0.5, 0.05),        # Min, Max, Step size (tuned)
    'learning_rate': (1e-5, 1e-2),           # Min, Max (log scale, tuned)
    'weight_decay': (1e-6, 1e-4),            # Min, Max (log scale, tuned)
    #'activation': ['relu', 'leakyrelu', 'tanh'],  # Categorical (tuned)

    # Training parameters that could also be tuned
    'batch_size': 64, #(32, 128, 32),             # Min, Max, Step size (tuned)
    'patience': (10, 30, 5),                  # Min, Max, Step size (tuned)
    'num_epochs': 100, #(10, 50, 5)                # Min, Max, Step size (tuned)
}


# Run hyperparameter optimization with model saving
trial_name = "vae_hyperparam_optimization_test"
pretrain_save_dir='/home/leilapirhaji/pretrained_models'
os.makedirs(pretrain_save_dir, exist_ok=True)

best_trial, study = optimize_hyperparameters(X_data_train, X_data_val, combined_params, trial_name, pretrain_save_dir, n_trials=20)


print(f"Best hyperparameters: {best_trial.params}")


[I 2024-09-17 22:41:34,615] A new study created in memory with name: no-name-4aeffad0-ab93-40e3-b375-08f498d7755a
/home/leilapirhaji/mz_embed_engine/ml/pretrain/train_pretrain_VAE.py:235: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.



Epoch 1/100, Train Loss: 219.3700
Validation Loss: 72.4860
Epoch 2/100, Train Loss: 72.5842
Validation Loss: 72.5135
Epoch 3/100, Train Loss: 72.5578
Validation Loss: 72.5552
Epoch 4/100, Train Loss: 72.5531
Validation Loss: 72.5473
Epoch 5/100, Train Loss: 72.5549
Validation Loss: 72.5642
Epoch 6/100, Train Loss: 72.5521
Validation Loss: 72.5770
Epoch 7/100, Train Loss: 72.5419
Validation Loss: 72.5420
Epoch 8/100, Train Loss: 72.5318
Validation Loss: 72.5467
Epoch 9/100, Train Loss: 72.5298
Validation Loss: 72.5328
Epoch 10/100, Train Loss: 72.5290
Validation Loss: 72.5263
Epoch 11/100, Train Loss: 72.5290
Validation Loss: 72.5234
Epoch 12/100, Train Loss: 72.5262
Validation Loss: 72.5237
Epoch 13/100, Train Loss: 72.5239
Validation Loss: 72.5149
Epoch 14/100, Train Loss: 72.5240
Validation Loss: 72.5176
Epoch 15/100, Train Loss: 72.5212
Validation Loss: 72.5190


[I 2024-09-17 22:42:05,740] Trial 0 finished with value: 72.50889104726363 and parameters: {'latent_size': 288, 'dropout_rate': 0.4613678111526134, 'learning_rate': 6.0115348199659396e-05, 'weight_decay': 3.6572068324510954e-05, 'patience': 15}. Best is trial 0 with value: 72.50889104726363.


Epoch 16/100, Train Loss: 72.5210
Validation Loss: 72.5089
Early stopping triggered at epoch 16
Encoder saved to /home/leilapirhaji/pretrained_models/vae_hyperparam_optimization_test/trial_0/encoder_state.pt
Hyperparameters saved to /home/leilapirhaji/pretrained_models/vae_hyperparam_optimization_test/trial_0/model_hyperparameters.json
Epoch 1/100, Train Loss: 37982496132777880.0000
Validation Loss: 13854.3767
Epoch 2/100, Train Loss: 26436690260618440704.0000
Validation Loss: 128.4619
Epoch 3/100, Train Loss: 128.4959
Validation Loss: 128.4672
Epoch 4/100, Train Loss: 128.4953
Validation Loss: 128.4714
Epoch 5/100, Train Loss: 128.5030
Validation Loss: 128.4627
Epoch 6/100, Train Loss: 128.4938
Validation Loss: 128.4677
Epoch 7/100, Train Loss: 128.5014
Validation Loss: 128.4647
Epoch 8/100, Train Loss: 128.5010
Validation Loss: 128.4633
Epoch 9/100, Train Loss: 128.5087
Validation Loss: 128.4658
Epoch 10/100, Train Loss: 128.5112
Validation Loss: 128.4785
Epoch 11/100, Train Loss: 12

[I 2024-09-17 22:46:10,131] Trial 1 finished with value: 128.41253911232462 and parameters: {'latent_size': 512, 'dropout_rate': 0.26184988135835274, 'learning_rate': 0.0006860042721194741, 'weight_decay': 1.4407267026324784e-05, 'patience': 25}. Best is trial 0 with value: 72.50889104726363.


...

Epoch 20/100, Train Loss: nan
Validation Loss: nan
Early stopping triggered at epoch 20
Encoder saved to /home/leilapirhaji/pretrained_models/vae_hyperparam_optimization_test/trial_17/encoder_state.pt
Hyperparameters saved to /home/leilapirhaji/pretrained_models/vae_hyperparam_optimization_test/trial_17/model_hyperparameters.json
Epoch 1/100, Train Loss: nan
Validation Loss: nan
Epoch 2/100, Train Loss: nan
Validation Loss: nan
Epoch 3/100, Train Loss: nan
Validation Loss: nan
Epoch 4/100, Train Loss: nan
Validation Loss: nan
Epoch 5/100, Train Loss: nan
Validation Loss: nan
Epoch 6/100, Train Loss: nan
Validation Loss: nan
Epoch 7/100, Train Loss: nan
Validation Loss: nan
Epoch 8/100, Train Loss: nan
Validation Loss: nan
Epoch 9/100, Train Loss: nan
Validation Loss: nan
Epoch 10/100, Train Loss: nan
Validation Loss: nan
Epoch 11/100, Train Loss: nan
Validation Loss: nan
Epoch 12/100, Train Loss: nan
Validation Loss: nan
Epoch 13/100, Train Loss: nan
Validation Loss: nan
Epoch 14/100, 

[W 2024-09-17 23:03:21,875] Trial 18 failed with parameters: {'latent_size': 256, 'dropout_rate': 0.11367955287729688, 'learning_rate': 0.005367047433888505, 'weight_decay': 1.0876766007866099e-06, 'patience': 20} because of the following error: The value nan is not acceptable.
[W 2024-09-17 23:03:21,876] Trial 18 failed with value nan.


Epoch 20/100, Train Loss: nan
Validation Loss: nan
Early stopping triggered at epoch 20
Encoder saved to /home/leilapirhaji/pretrained_models/vae_hyperparam_optimization_test/trial_18/encoder_state.pt
Hyperparameters saved to /home/leilapirhaji/pretrained_models/vae_hyperparam_optimization_test/trial_18/model_hyperparameters.json
Epoch 1/100, Train Loss: nan
Validation Loss: nan
Epoch 2/100, Train Loss: nan
Validation Loss: nan
Epoch 3/100, Train Loss: nan
Validation Loss: nan
Epoch 4/100, Train Loss: nan
Validation Loss: nan
Epoch 5/100, Train Loss: nan
Validation Loss: nan
Epoch 6/100, Train Loss: nan
Validation Loss: nan
Epoch 7/100, Train Loss: nan
Validation Loss: nan
Epoch 8/100, Train Loss: nan
Validation Loss: nan
Epoch 9/100, Train Loss: nan
Validation Loss: nan
Epoch 10/100, Train Loss: nan
Validation Loss: nan
Epoch 11/100, Train Loss: nan
Validation Loss: nan
Epoch 12/100, Train Loss: nan
Validation Loss: nan
Epoch 13/100, Train Loss: nan
Validation Loss: nan
Epoch 14/100, 

[W 2024-09-17 23:04:00,661] Trial 19 failed with parameters: {'latent_size': 256, 'dropout_rate': 0.10957649478312234, 'learning_rate': 0.004956462027353261, 'weight_decay': 1.066467539793574e-06, 'patience': 20} because of the following error: The value nan is not acceptable.
[W 2024-09-17 23:04:00,662] Trial 19 failed with value nan.


Epoch 20/100, Train Loss: nan
Validation Loss: nan
Early stopping triggered at epoch 20
Encoder saved to /home/leilapirhaji/pretrained_models/vae_hyperparam_optimization_test/trial_19/encoder_state.pt
Hyperparameters saved to /home/leilapirhaji/pretrained_models/vae_hyperparam_optimization_test/trial_19/model_hyperparameters.json
Best trial: FrozenTrial(number=2, state=TrialState.COMPLETE, values=[64.45824883908642], datetime_start=datetime.datetime(2024, 9, 17, 22, 46, 10, 132333), datetime_complete=datetime.datetime(2024, 9, 17, 22, 46, 40, 455189), params={'latent_size': 256, 'dropout_rate': 0.1489539827766307, 'learning_rate': 0.00017595104205543589, 'weight_decay': 1.219999696915712e-06, 'patience': 15}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'latent_size': IntDistribution(high=512, log=False, low=256, step=32), 'dropout_rate': FloatDistribution(high=0.5, log=False, low=0.1, step=None), 'learning_rate': FloatDistribution(high=0.01, log=True, low=1e-

In [47]:
study.best_trial.value, study.best_trial.params

(64.45824883908642,
 {'latent_size': 256,
  'dropout_rate': 0.1489539827766307,
  'learning_rate': 0.00017595104205543589,
  'weight_decay': 1.219999696915712e-06,
  'patience': 15})

In [48]:
try:
    import optuna.visualization as vis
    vis.plot_optimization_history(study).show()
    vis.plot_param_importances(study).show()
except ImportError:
    print("Optuna visualization is not available. Install optuna[visualization] for plotting.")
